In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import urllib.request
import urllib.request
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tqdm import tqdm
import os

print('-------------------')
print('|     lab1         |')
print('-------------------')

if os.path.exists('/semi.csv'):
    '''
    혹시 파일이 생기지 않는다면, 아래 두 줄의 스크립스틑 파이썬 .py파일로 만들어서 실행하면 됩니다. 
    '''
    url = "https://drive.google.com/uc?export=download&id=1XCU0eo2xZ03xhxJhdrCnVjduCoaBQ7kJ"
    urllib.request.urlretrieve(url, "semi.csv")  # save in a file
else:
    print('data already exist')


-------------------
|     lab1         |
-------------------
data already exist


In [60]:
df = pd.read_csv('semi.csv')

In [61]:
df.isnull().sum()

Time          0
0             6
1             7
2            14
3            14
             ..
586           1
587           1
588           1
589           1
Pass/Fail     0
Length: 592, dtype: int64

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 592 entries, Time to Pass/Fail
dtypes: float64(590), int64(1), object(1)
memory usage: 7.1+ MB


In [63]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,581,582,583,584,585,586,587,588,589,Pass/Fail
count,1561.000000,1560.000000,1553.000000,1553.000000,1553.000000,1553.0,1553.000000,1558.000000,1565.000000,1565.000000,...,618.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1566.000000,1567.000000
mean,3014.452896,2495.850231,2200.547318,1396.376627,4.197013,100.0,101.112908,0.121822,1.462862,-0.000841,...,97.934373,0.500096,0.015318,0.003847,3.067826,0.021458,0.016475,0.005283,99.670066,-0.867262
std,73.621787,80.407705,29.513152,441.691640,56.355540,0.0,6.237214,0.008961,0.073897,0.015116,...,87.520966,0.003404,0.017180,0.003720,3.578033,0.012358,0.008808,0.002867,93.891919,0.498010
min,2743.240000,2158.750000,2060.660000,0.000000,0.681500,100.0,82.131100,0.000000,1.191000,-0.053400,...,0.000000,0.477800,0.006000,0.001700,1.197500,-0.016900,0.003200,0.001000,0.000000,-1.000000
25%,2966.260000,2452.247500,2181.044400,1081.875800,1.017700,100.0,97.920000,0.121100,1.411200,-0.010800,...,46.184900,0.497900,0.011600,0.003100,2.306500,0.013425,0.010600,0.003300,44.368600,-1.000000
50%,3011.490000,2499.405000,2201.066700,1285.214400,1.316800,100.0,101.512200,0.122400,1.461600,-0.001300,...,72.288900,0.500200,0.013800,0.003600,2.757650,0.020500,0.014800,0.004600,71.900500,-1.000000
75%,3056.650000,2538.822500,2218.055500,1591.223500,1.525700,100.0,104.586700,0.123800,1.516900,0.008400,...,116.539150,0.502375,0.016500,0.004100,3.295175,0.027600,0.020300,0.006400,114.749700,-1.000000
max,3356.350000,2846.440000,2315.266700,3715.041700,1114.536600,100.0,129.252200,0.128600,1.656400,0.074900,...,737.304800,0.509800,0.476600,0.104500,99.303200,0.102800,0.079900,0.028600,737.304800,1.000000


In [64]:

def NullHandler(df: pd.DataFrame):
    features = range(590)
    for fe in features:
        if df[str(fe)].isnull().sum() != 0:
            df[str(fe)] = df[str(fe)].fillna(np.mean(df[str(fe)]))
    assert df.isnull().sum().sum() == 0
    print('NULL PASSED!\n')
    return df


def DuplicatedHandler(df: pd.DataFrame):
    features = range(590)
    stack = []
    idx = []
    for i,fe in enumerate(features):
        st = df[str(fe)].std()
        me = df[str(fe)].mean()
        mx = df[str(fe)].max()
        mn = df[str(fe)].mean()
        if (st, me, mx, mn) in stack:
            idx.append(fe)
            print('duplicated!',i)
        else:
            stack.append((st, me, mx, mn))

    for i in idx:
        df = df.drop(str(i), axis=1)
    df = df.drop(['Time'], axis=1)
    return df


def Norm(df: pd.DataFrame):
    numeric_features = df.columns[:-1]

    for feature in numeric_features:
        sk = StandardScaler()
        df[feature] = sk.fit_transform(df[[feature]])
    return df


def DataHandler(df: pd.DataFrame):
    df = NullHandler(df)
    df = DuplicatedHandler(df)
    df = Norm(df)
    return df


df = DataHandler(df)

NULL PASSED!

duplicated! 52
duplicated! 69
duplicated! 97
duplicated! 141
duplicated! 149
duplicated! 178
duplicated! 179
duplicated! 186
duplicated! 189
duplicated! 190
duplicated! 191
duplicated! 192
duplicated! 193
duplicated! 194
duplicated! 226
duplicated! 229
duplicated! 230
duplicated! 231
duplicated! 232
duplicated! 233
duplicated! 234
duplicated! 235
duplicated! 236
duplicated! 237
duplicated! 240
duplicated! 241
duplicated! 242
duplicated! 243
duplicated! 256
duplicated! 257
duplicated! 258
duplicated! 259
duplicated! 260
duplicated! 261
duplicated! 262
duplicated! 263
duplicated! 264
duplicated! 265
duplicated! 266
duplicated! 276
duplicated! 284
duplicated! 313
duplicated! 314
duplicated! 315
duplicated! 322
duplicated! 325
duplicated! 326
duplicated! 327
duplicated! 328
duplicated! 329
duplicated! 330
duplicated! 364
duplicated! 369
duplicated! 370
duplicated! 371
duplicated! 372
duplicated! 373
duplicated! 374
duplicated! 375
duplicated! 378
duplicated! 379
duplicated! 3

In [1]:
print('is NaN data check : ', df.isna().sum().sum())


NameError: name 'df' is not defined

In [66]:
df

,0,1,2,3,4,5,6,7,8,9,...,581,582,583,584,585,586,587,588,589,Pass/Fail
0,0.224309,0.849725,-0.436273,0.033555,-0.050580,0.0,-0.563790,0.266269,0.509826,1.128417,...,2.587617e-16,0.118699,-0.204890,-0.093207,-0.197113,0.000000,0.000000,-3.027186e-16,1.514500e-16,-1
1,1.107136,-0.382910,1.017137,0.153067,-0.060045,0.0,0.198217,0.322244,0.456999,0.022582,...,2.007880e+00,0.530203,0.406679,0.444706,0.385059,-0.960174,0.411853,2.501244e-01,1.156689e+00,-1
2,-1.114158,0.799102,-0.481289,0.686213,-0.047906,0.0,-0.906210,0.255074,-0.260907,0.327183,...,-2.744816e-01,-1.262780,0.022264,0.014375,0.029833,2.991151,3.627063,3.321419e+00,-1.791486e-01,1
3,-0.350312,-0.198875,-0.051547,-1.106948,-0.051290,0.0,0.503246,-0.013602,0.343218,-0.765408,...,-4.386698e-01,-0.322199,-0.292257,-0.362164,-0.283417,-0.101895,-0.178927,-3.082928e-01,-2.752459e-01,-1
4,0.242143,0.087526,1.117387,-0.158919,-0.047492,0.0,-0.115382,0.187905,0.545044,-0.149584,...,2.587617e-16,-5.906899,26.867231,27.071425,26.913347,-0.101895,-0.178927,-3.082928e-01,-2.752459e-01,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,-1.566122,-0.392636,-0.708645,3.842373,-0.048368,0.0,-3.039363,0.333438,-1.631702,-0.242289,...,1.916244e+00,-0.380985,-0.059279,0.014375,-0.056191,-1.186890,-0.303900,-2.035896e-01,1.103056e+00,-1
1563,0.515363,0.332906,-0.067433,-0.618139,-0.059208,0.0,-0.425952,-0.147940,-0.400425,-0.348237,...,2.587617e-16,-0.763096,-0.129172,-0.066312,-0.124177,-1.186890,-0.303900,-2.035896e-01,1.103056e+00,-1
1564,-0.485220,-1.447220,0.195859,-0.650359,-0.060148,0.0,-0.273986,-0.114356,0.000000,0.000000,...,-9.907605e-01,-0.410378,-0.001034,0.068167,-0.002468,-0.142380,-0.894681,-9.714133e-01,-5.983777e-01,-1
1565,-1.627246,0.450858,-0.800571,-0.483761,-0.046793,0.0,-0.372966,-0.058381,-0.008962,-0.421077,...,-8.085176e-02,0.089306,0.144578,-0.012520,0.139376,0.383924,0.911745,7.736405e-01,-6.581942e-02,-1


In [67]:
feature = df.drop('Pass/Fail', axis=1)
target = df['Pass/Fail']

In [68]:
target

0      -1
1      -1
2       1
3      -1
4      -1
       ..
1562   -1
1563   -1
1564   -1
1565   -1
1566   -1
Name: Pass/Fail, Length: 1567, dtype: int64

In [69]:
feature

,0,1,2,3,4,5,6,7,8,9,...,580,581,582,583,584,585,586,587,588,589
0,0.224309,0.849725,-0.436273,0.033555,-0.050580,0.0,-0.563790,0.266269,0.509826,1.128417,...,4.436053e-16,2.587617e-16,0.118699,-0.204890,-0.093207,-0.197113,0.000000,0.000000,-3.027186e-16,1.514500e-16
1,1.107136,-0.382910,1.017137,0.153067,-0.060045,0.0,0.198217,0.322244,0.456999,0.022582,...,3.089342e-01,2.007880e+00,0.530203,0.406679,0.444706,0.385059,-0.960174,0.411853,2.501244e-01,1.156689e+00
2,-1.114158,0.799102,-0.481289,0.686213,-0.047906,0.0,-0.906210,0.255074,-0.260907,0.327183,...,4.809624e+00,-2.744816e-01,-1.262780,0.022264,0.014375,0.029833,2.991151,3.627063,3.321419e+00,-1.791486e-01
3,-0.350312,-0.198875,-0.051547,-1.106948,-0.051290,0.0,0.503246,-0.013602,0.343218,-0.765408,...,-5.093731e-01,-4.386698e-01,-0.322199,-0.292257,-0.362164,-0.283417,-0.101895,-0.178927,-3.082928e-01,-2.752459e-01
4,0.242143,0.087526,1.117387,-0.158919,-0.047492,0.0,-0.115382,0.187905,0.545044,-0.149584,...,4.436053e-16,2.587617e-16,-5.906899,26.867231,27.071425,26.913347,-0.101895,-0.178927,-3.082928e-01,-2.752459e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,-1.566122,-0.392636,-0.708645,3.842373,-0.048368,0.0,-3.039363,0.333438,-1.631702,-0.242289,...,-3.559405e-01,1.916244e+00,-0.380985,-0.059279,0.014375,-0.056191,-1.186890,-0.303900,-2.035896e-01,1.103056e+00
1563,0.515363,0.332906,-0.067433,-0.618139,-0.059208,0.0,-0.425952,-0.147940,-0.400425,-0.348237,...,4.436053e-16,2.587617e-16,-0.763096,-0.129172,-0.066312,-0.124177,-1.186890,-0.303900,-2.035896e-01,1.103056e+00
1564,-0.485220,-1.447220,0.195859,-0.650359,-0.060148,0.0,-0.273986,-0.114356,0.000000,0.000000,...,-1.481113e+00,-9.907605e-01,-0.410378,-0.001034,0.068167,-0.002468,-0.142380,-0.894681,-9.714133e-01,-5.983777e-01
1565,-1.627246,0.450858,-0.800571,-0.483761,-0.046793,0.0,-0.372966,-0.058381,-0.008962,-0.421077,...,1.076097e+00,-8.085176e-02,0.089306,0.144578,-0.012520,0.139376,0.383924,0.911745,7.736405e-01,-6.581942e-02


In [70]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, \
                                                    test_size=0.2, random_state=11, stratify=target)

param_grid = {
    'n_estimators': [200, 400],
    'learning_rate': [0.01, 0.05],
    'max_depth': [5, 7],
    'subsample': [0.8,1.0],
    'colsample_bytree': [0.7,1.0],
    'gamma': [0,0.2],
}

y_train = [0 if label == -1 else 1 for label in y_train]
y_test = [0 if label == -1 else 1 for label in y_test]

In [71]:
'''
학습시간이 오래 걸리니 해당 셀은 신중하게 실행하십시오.
'''
import os
import joblib

if os.path.exists('/best_xgboost_model.pkl'):
    best_model = joblib.load('best_xgboost_model.pkl')
else:
    from sklearn.utils.class_weight import compute_class_weight
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    model = xgb.XGBClassifier(eval_metric='auc', tree_method='hist', scale_pos_weight=class_weights[1])
    grid_search = GridSearchCV(model, param_grid, cv=2, scoring='balanced_accuracy', verbose=2)
    grid_search.fit(X_train, y_train)
    print("Best parameters: ", grid_search.best_params_)
    best_model = grid_search.best_estimator_

Fitting 2 folds for each of 64 candidates, totalling 128 fits
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.01, max_depth=5, n_estimators=200, subsample=0.8; total time=   1.5s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.01, max_depth=5, n_estimators=200, subsample=0.8; total time=   1.0s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.01, max_depth=5, n_estimators=200, subsample=1.0; total time=   1.3s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.01, max_depth=5, n_estimators=200, subsample=1.0; total time=   1.2s
[CV] END colsample_bytree=0.7, gamma=0, learning_rate=0.01, max_depth=5, n_estimators=400, subsample=0.8; total time=   3.2s


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import f1_score, balanced_accuracy_score

f1 = f1_score(best_model.predict(X_test), y_test)
balance = balanced_accuracy_score(best_model.predict(X_test), y_test)
print(f'f1 score is : {f1}')
print(f'balanced score is : {balance}')

In [ ]:
import joblib

joblib.dump(best_model, 'best_xgboost_model.pkl')

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

best_model = joblib.load('best_xgboost_model.pkl')

y_pred = best_model.predict(X_test)
f1 = f1_score(y_test, y_pred, average='binary')
balance = balanced_accuracy_score(y_pred, y_test)
print(f'f1 score is : {f1}')
print(f'balanced score is : {balance}')